# Random Forest progression prediction on exome sequencing

In [25]:
# importing useful package
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle 
import random



from sklearn.ensemble import RandomForestClassifier,ExtraTreesRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

import preprocessing

## Training a model on our data

In [34]:
All_MS = pd.read_csv("WES_wide_NAfrequency.csv")



dt_predict = pd.read_csv('mutation_profile_pval05.txt', sep="\t",usecols=['Row.names','Cohort',"ipfs","ipfsev"])
dt_predict.index = dt_predict['Row.names']

prefilt_thr = filtcoefNA = 0.05
prefilt = "NA"

dt = pd.read_csv('./mutation_profile_pval05.txt', sep="\t")
dt.index = dt['Row.names']

# Only keep MS and ipfs
dt = dt.filter(regex='chr|ipfs|Cohort')

# Creating the time and event columns
time_column = 'ipfs'
event_column = 'ipfsev'


### Filtering of MS and imputation #####

# on Nipicol : filterout NA and impute
dt_tree_NIPICOL,features_NIPICOL = preprocessing.clean_NA_duplicate_input(dt[dt['Cohort']=='NIPICOL'],filtcoef=filtcoefNA)


lowNA = All_MS[All_MS['NaFreq']<prefilt_thr]

# filter out MS usually miss covered on other cohortes
dt_tree_NIPICOL = dt_tree_NIPICOL[dt_tree_NIPICOL.columns.intersection(lowNA['hg38chridNIPICOLformat'].append(pd.Series(['Cohort','ipfs','ipfsev'])))]

#keep only nipicol chrid
dt_tree_ImmunoMSI = dt[dt_tree_NIPICOL.columns]
#impute missing data on ImmunoMSI
dt_tree_ImmunoMSI,features_ImmunoMSI = preprocessing.clean_NA_duplicate_input(dt_tree_ImmunoMSI[dt_tree_ImmunoMSI['Cohort']=='RICKI'],filtcoef=1)


# intersection of NIPICOL and ImmunoMSI to keep only shared position after filtering and imputation
shared_features = set(features_ImmunoMSI).intersection(set(features_NIPICOL))
shared_features_ipfs = shared_features.union({time_column,event_column,'Cohort'})
        

ipfs_cohorte = dt[['ipfs','ipfsev','Cohort']]


dt_tree = dt_tree_ImmunoMSI[shared_features_ipfs].append(dt_tree_NIPICOL[shared_features_ipfs])

# resample Nipicol cohorte             
X_train, E_train, X_test, E_test = preprocessing.upsampling(dt_tree,1000)
   
# Making sure features are in the same order
X_train = X_train[shared_features]
X_test = X_test[shared_features]
X = dt_tree[shared_features]

### Random Forest ####
clf_forest = RandomForestClassifier( n_estimators=2000, max_features=1)
clf_forest.fit(X_train,E_train.astype(int))

# save the model to disk
# NB the random state of the model presented in the article was not fixed which may explain difference at the end. A version of it is saved under the name RFclassifier.sav and can be load
filename = './New_RFclassifier.sav'
pickle.dump(clf_forest, open(filename, 'wb'))


dt_predict['RF_prediction'] = pd.DataFrame(clf_forest.predict_proba(X),index=X.index)[1]


dt_predict.to_csv('./predicted_risks.csv')

/tmp/ipykernel_114152/4215150855.py:31: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dt_tree_NIPICOL = dt_tree_NIPICOL[dt_tree_NIPICOL.columns.intersection(lowNA['hg38chridNIPICOLformat'].append(pd.Series(['Cohort','ipfs','ipfsev'])))]
/home/aurelien/anaconda3/lib/python3.8/site-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
/tmp/ipykernel_114152/4215150855.py:47: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dt_tree = dt_tree_ImmunoMSI[shared_features_ipfs].append(dt_tree_NIPICOL[shared_features_ipfs])
/tmp/ipykernel_114152/4215150855.py:47: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dt_tree = dt_tree_ImmunoMSI[shared_features_ipfs].append(dt_tree_NIPICOL[shared_featur

## Model validation using Cox

In [39]:
from lifelines import CoxPHFitter
import math

import kaplanmeier as km

time_col = "ipfs"
event_col = 'ipfsev'

cph = CoxPHFitter()
cph.fit(dt_predict[dt_predict["Cohort"]=="RICKI"][['RF_prediction',time_col,event_col]],time_col, event_col=event_col)

<lifelines.CoxPHFitter: fitted with 66 total observations, 54 right-censored observations>

In [41]:
cph.summary

,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p)
covariate,,,,,,,,,,
RF_prediction,8.031809,3077.302378,2.347655,3.43049,12.633128,30.891765,306547.394314,3.421205,0.000623,10.647454
